In [1]:
# import sys
# !{sys.executable} -m pip install spacy

  Using cached tqdm-4.64.1-py2.py3-none-any.whl (78 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached typing_extensions-4.4.0-py3-none-any.whl (26 kB)
  Using cached certifi-2022.12.7-py3-none-any.whl (155 kB)
  Using cached urllib3-1.26.13-py2.py3-none-any.whl (140 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached click-8.1.3-py3-none-any.whl (96 kB)


You should consider upgrading via the 'c:\users\chin\desktop\stuff\projects\walnut-ai-assignment\venv\scripts\python.exe -m pip install --upgrade pip' command.


## Reading text file

In [1]:
import pandas as pd
import numpy as np

In [2]:
filename = './restauranttrain.bio.txt'
df = pd.read_table(filename, sep='\t', header=None, names=['label','text'], skip_blank_lines=False)
df.head(20)

,label,text
0,B-Rating,2
1,I-Rating,start
2,O,restaurants
3,O,with
4,B-Amenity,inside
5,I-Amenity,dining
6,NaN,NaN
7,O,34
8,NaN,NaN
9,B-Rating,5


## Preprocessing Data

In [3]:
train = []
sent_tokens = []
entities = []
start_position = 0
position = 0
entity = None
for index, (label, text) in df.iterrows():
#     print(label, text)
    if type(label)!= str and type(text)!= str:
#         print('break!')
#         print(sent_tokens)
        if entity:
            entities.append((start_position, position-1, entity))
            entity = None
        train.append((' '.join(sent_tokens),{"entities":entities}))
        sent_tokens = []
        entities = []
        position = 0
        entity = None
        continue
    sent_tokens.append(text)
    if label[0] == 'O' and entity is not None:
        entities.append((start_position, position-1, entity))  
        entity = None
    elif label[0] == 'B':
        if entity:
            entities.append((start_position, position-1, entity))  
            entity = None
        entity = label.replace('B-','')
        start_position = position
    position += len(text) + 1
train
    

[('2 start restaurants with inside dining',
  {'entities': [(0, 7, 'Rating'), (25, 38, 'Amenity')]}),
 ('34', {'entities': []}),
 ('5 star resturants in my town',
  {'entities': [(0, 6, 'Rating'), (18, 28, 'Location')]}),
 ('98 hong kong restaurant reasonable prices',
  {'entities': [(3, 12, 'Restaurant_Name'), (24, 34, 'Price')]}),
 ('a great lunch spot but open till 2 a m passims kitchen',
  {'entities': [(23, 38, 'Hours'), (39, 54, 'Restaurant_Name')]}),
 ('a place that serves soft serve ice cream',
  {'entities': [(20, 40, 'Dish')]}),
 ('a restaurant that is good for groups',
  {'entities': [(21, 25, 'Rating'), (26, 36, 'Amenity')]}),
 ('a salad would make my day', {'entities': [(2, 7, 'Dish')]}),
 ('a smoothie would hit the spot', {'entities': [(2, 10, 'Cuisine')]}),
 ('a steak would be nice', {'entities': [(2, 7, 'Dish')]}),
 ('a sub place near independence boulevard',
  {'entities': [(2, 5, 'Dish'), (12, 39, 'Location')]}),
 ('about how much is a midpriced bottle of good wine at

## Model Training

In [24]:
import spacy
# spacy.cli.download("en_core_web_sm")

[i] Saving to output directory: output
[i] Using CPU

=========================== Initializing pipeline ===========================
[+] Initialized pipeline

============================= Training pipeline =============================
[i] Pipeline: ['tok2vec', 'ner']
[i] Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     59.79    0.00    0.00    0.00    0.00
  0     200        201.69   5746.63   55.13   60.35   50.75    0.55
  0     400        507.80   4637.54   69.54   73.36   66.11    0.70
  1     600        707.55   4744.58   74.40   75.73   73.12    0.74
  1     800        804.80   4880.25   79.19   80.38   78.03    0.79
  2    1000        938.98   5364.04   81.53   82.19   80.88    0.82
  3    1200       1118.66   5867.45   83.76   84.37   83.16    0.84
  4    1400       1346.89   6247.62   85.35   85.28   85.43    0.85
  5    1600       173

[2023-01-01 23:11:01,149] [INFO] Set up nlp object from config
[2023-01-01 23:11:01,158] [INFO] Pipeline: ['tok2vec', 'ner']
[2023-01-01 23:11:01,161] [INFO] Created vocabulary
[2023-01-01 23:11:01,161] [INFO] Finished initializing nlp object
[2023-01-01 23:11:02,775] [INFO] Initialized pipeline components: ['tok2vec', 'ner']


In [5]:
from spacy.tokens import DocBin

In [6]:
db = DocBin() # create a DocBin object

In [7]:
nlp = spacy.load("en_core_web_sm")
for text, annot in train: # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
#     print(doc)
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
#         print(span)
        if span is None:
            print("Skipping entity")
            pass
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    print(doc, doc.ents)
    db.add(doc)

2 start restaurants with inside dining (2 start, inside dining)
34 ()
5 star resturants in my town (5 star, in my town)
98 hong kong restaurant reasonable prices (hong kong, reasonable)
a great lunch spot but open till 2 a m passims kitchen (open till 2 a m, passims kitchen)
a place that serves soft serve ice cream (soft serve ice cream,)
a restaurant that is good for groups (good, for groups)
a salad would make my day (salad,)
a smoothie would hit the spot (smoothie,)
a steak would be nice (steak,)
a sub place near independence boulevard (sub, near independence boulevard)
about how much is a midpriced bottle of good wine at davidos italian palace (midpriced, bottle, good, wine, davidos italian palace)
about how much will it cost to eat at johns pizza cafe for dinner per person (cost, johns pizza cafe, dinner, per person)
adress of the nearest cracker barrel (nearest, cracker barrel)
after hour dining (after hour dining,)
all i want is a bagel and coffee can you direct me (bagel and co

In [8]:
db.to_disk("./train.spacy")

In [11]:
!python -m spacy init fill-config base_config.cfg config.cfg

[+] Auto-filled config with all values
[+] Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [12]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy

^C


In [13]:
ner = spacy.load(r"./output/model-last") #load the final model

In [30]:
doc = ner("This is an excellent 5 star fish and chips restaurant for fine dining and it is near to my house.")
entities = {}
for ent in doc.ents:
    entities[ent.text] = ent.label_
entities

{'5 star': 'Rating',
 'fish and chips': 'Cuisine',
 'fine dining': 'Amenity',
 'near to my house.': 'Location'}

## Usage

In [32]:
!python ner_predict.py "This is an excellent 5 star fish and chips restaurant for fine dining and it is near to my house."

{   '5 star': 'Rating',
    'fine dining': 'Amenity',
    'fish and chips': 'Cuisine',
    'near to my house': 'Location'}
